In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem

import warnings

warnings.filterwarnings("ignore")

In [2]:
suppl = Chem.SDMolSupplier('/Users/adityadey/Downloads/Compound_000000001_000500000.sdf')

len(suppl)

444049

In [3]:
# suppl[0].GetProp("PUBCHEM_COMPOUND_CID")

properties = suppl[0].GetPropNames()

## Available Properties that can be extracted.
for x in properties:
    print(x)

PUBCHEM_COMPOUND_CID
PUBCHEM_COMPOUND_CANONICALIZED
PUBCHEM_CACTVS_COMPLEXITY
PUBCHEM_CACTVS_HBOND_ACCEPTOR
PUBCHEM_CACTVS_HBOND_DONOR
PUBCHEM_CACTVS_ROTATABLE_BOND
PUBCHEM_CACTVS_SUBSKEYS
PUBCHEM_IUPAC_OPENEYE_NAME
PUBCHEM_IUPAC_CAS_NAME
PUBCHEM_IUPAC_NAME_MARKUP
PUBCHEM_IUPAC_NAME
PUBCHEM_IUPAC_SYSTEMATIC_NAME
PUBCHEM_IUPAC_TRADITIONAL_NAME
PUBCHEM_IUPAC_INCHI
PUBCHEM_IUPAC_INCHIKEY
PUBCHEM_XLOGP3_AA
PUBCHEM_EXACT_MASS
PUBCHEM_MOLECULAR_FORMULA
PUBCHEM_MOLECULAR_WEIGHT
PUBCHEM_OPENEYE_CAN_SMILES
PUBCHEM_OPENEYE_ISO_SMILES
PUBCHEM_CACTVS_TPSA
PUBCHEM_MONOISOTOPIC_WEIGHT
PUBCHEM_TOTAL_CHARGE
PUBCHEM_HEAVY_ATOM_COUNT
PUBCHEM_ATOM_DEF_STEREO_COUNT
PUBCHEM_ATOM_UDEF_STEREO_COUNT
PUBCHEM_BOND_DEF_STEREO_COUNT
PUBCHEM_BOND_UDEF_STEREO_COUNT
PUBCHEM_ISOTOPIC_ATOM_COUNT
PUBCHEM_COMPONENT_COUNT
PUBCHEM_CACTVS_TAUTO_COUNT
PUBCHEM_COORDINATE_TYPE
PUBCHEM_BONDANNOTATIONS


In [4]:
data = {"PUBCHEM_COMPOUND_CID": [],
        "PUBCHEM_OPENEYE_CAN_SMILES": [],
        "PUBCHEM_OPENEYE_ISO_SMILES": []}

for mol in suppl:
    if mol != None:
        for key in data.keys():
            data[key].append(mol.GetProp(key))

[01:39:40] WARNING: not removing hydrogen atom without neighbors
[01:39:43] Explicit valence for atom # 0 Cl, 7, is greater than permitted
[01:39:43] ERROR: Could not sanitize molecule ending on line 4301805
[01:39:43] ERROR: Explicit valence for atom # 0 Cl, 7, is greater than permitted
[01:39:43] Explicit valence for atom # 0 Br, 3, is greater than permitted
[01:39:43] ERROR: Could not sanitize molecule ending on line 4346502
[01:39:43] ERROR: Explicit valence for atom # 0 Br, 3, is greater than permitted
[01:39:43] Explicit valence for atom # 0 Br, 5, is greater than permitted
[01:39:43] ERROR: Could not sanitize molecule ending on line 4347777
[01:39:43] ERROR: Explicit valence for atom # 0 Br, 5, is greater than permitted
[01:39:43] Explicit valence for atom # 0 Cl, 3, is greater than permitted
[01:39:43] ERROR: Could not sanitize molecule ending on line 4351511
[01:39:43] ERROR: Explicit valence for atom # 0 Cl, 3, is greater than permitted
[01:39:43] WARNING: not removing hydrog

In [5]:

## Convert to dataframe.
pubchem = pd.DataFrame(data).set_index("PUBCHEM_COMPOUND_CID")

## Rename columns.
pubchem.rename(columns={"PUBCHEM_OPENEYE_CAN_SMILES":"CANONICAL_SMILES", 
               "PUBCHEM_OPENEYE_ISO_SMILES": "ISOMERIC_SMILES"},
               inplace=True)
pubchem.head(3)

,CANONICAL_SMILES,ISOMERIC_SMILES
PUBCHEM_COMPOUND_CID,,
1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
3,C1=CC(C(C(=C1)C(=O)O)O)O,C1=CC(C(C(=C1)C(=O)O)O)O


In [6]:
print(f"Unreadable Molecules: {len(suppl) - pubchem.shape[0]}\n")
pubchem.info()

Unreadable Molecules: 565

<class 'pandas.core.frame.DataFrame'>
Index: 443484 entries, 1 to 500000
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   CANONICAL_SMILES  443484 non-null  object
 1   ISOMERIC_SMILES   443484 non-null  object
dtypes: object(2)
memory usage: 10.2+ MB


In [7]:
## This shows isomeric smiles are unique.
pubchem.nunique()

CANONICAL_SMILES    425602
ISOMERIC_SMILES     443393
dtype: int64

SMILES denotes a molecular structure as a graph with optional chiral indications. This is essentially the two-dimensional picture chemists draw to describe a molecule. SMILES describing only the labeled molecular graph (i.e. atoms and bonds, but no chiral or isotopic information) are known as generic SMILES. There are usually a large number of valid generic SMILES which represent a given structure. A canonicalization algorithm exists to generate one special generic SMILES among all valid possibilities; this special one is known as the "unique SMILES". SMILES written with isotopic and chiral specifications are collectively known as "isomeric SMILES". A unique isomeric SMILES is known as an "absolute SMILES". See the following examples.

|Input SMILES|	Unique SMILES|
|---|---|
|OCC|CCO|
|[CH3][CH2][OH]	|CCO|
|C-C-O	|CCO|
|C(O)C	|CCO|

https://ics.uci.edu/~dock/manuals/DaylightTheoryManual/theory.smiles.html